In [ ]:
# import sys
# !{sys.executable} -m pip install pip earthengine-api
# !{sys.executable} -m pip install pip geemap
# !{sys.executable} -m pip install pip wget

In [ ]:
import os, requests, json, geojson
import pandas as pd
import geopandas as gpd
import boto3
#import ee
#import geemap
import fiona
import shapely

In [ ]:
# define directory
out_dir = os.getcwd()
bucket_name = 'cities-indicators'
aws_s3_dir = "https://"+bucket_name+".s3.eu-west-3.amazonaws.com"
boundary_ext = '/data/boundaries/'
indicators_file_aws = 'indicators/indicators.csv'

In [ ]:
OUTPUT_FILENAME = 'LND-6-protected-areas.csv'

In [ ]:
WDPA_FILENAME = 'data/WDPA_data/WDPA_Oct2022_Public_shp-polygons'
WDPA_TEMPFILENAME = 'temp_wdpadata'

In [ ]:
# connect to s3
aws_credentials = pd.read_csv('/home/jovyan/PlanetaryComputerExamples/aws_credentials.csv')
aws_key = aws_credentials.iloc[0]['Access key ID']
aws_secret = aws_credentials.iloc[0]['Secret access key']

s3 = boto3.resource(
    service_name='s3',
    aws_access_key_id=aws_key,
    aws_secret_access_key=aws_secret
)

In [ ]:
source_bucket = s3.Bucket(bucket_name)
for ext in ('shp', 'shx'):
    source_bucket.download_file('{0}.{1}'.format(WDPA_FILENAME, ext), '{0}.{1}'.format(WDPA_TEMPFILENAME, ext))

In [ ]:
wdpa_multi = shapely.geometry.MultiPolygon([shapely.geometry.shape(poly['geometry']) for poly in fiona.open('{0}.shp'.format(WDPA_TEMPFILENAME))])

In [ ]:
# get list of cities
boundary_georef = pd.read_csv(aws_s3_dir + boundary_ext + 'boundary_georef.csv')
boundary_georef

In [ ]:
def do_one_geom(row):
    poly = row[0]
    box = poly.bounds
    protected_box = shapely.ops.clip_by_rect(wdpa_multi, *box)
    protected_box = shapely.ops.unary_union(protected_box)    # unary_union repairs inalid geometries

    protected_poly = protected_box.intersection(poly)
    protected_area = protected_poly.area
    result = protected_area/poly.area
    print('{0}: {1}'.format(row[1], result))
    return result

In [ ]:
for i in range(0,len(boundary_georef)):
    if not OUTPUT_FILENAME in os.listdir('.'):
        so_far_df = pd.DataFrame()
        so_far_df.to_csv(OUTPUT_FILENAME)
        so_far = []
    else:
        so_far_df = pd.read_csv(OUTPUT_FILENAME)
        so_far = [so_far_df.iloc[j]['geo_id'] for j in range(len(so_far_df))]
    
    most_recent = []
    #outputs_df = pd.DataFrame()
    do_units = True
    for boundary_name in ['aoi_boundary_name', 'units_boundary_name']:
        if type(boundary_georef.loc[i, boundary_name]) != float: # sometimes boundary_id is nan
            boundary_id = boundary_georef.loc[i, 'geo_name']+'-' + boundary_georef.loc[i, boundary_name]
            boundary_path = aws_s3_dir + boundary_ext +'boundary-'+boundary_id+'.geojson'
            boundary_geo = requests.get(boundary_path).json()
            temp_gdf = gpd.GeoDataFrame.from_features(boundary_geo)
            if (boundary_name == 'aoi_boundary_name') or (boundary_name == 'units_boundary_name' and do_units):
                temp_gdf['LND_6_percentProtectedArea'] = temp_gdf.apply(do_one_geom, axis=1)
            else:
                 temp_gdf['LND_6_percentProtectedArea'] = temp_gdf.apply(lambda x: 0, axis=1)
            if boundary_name == 'aoi_boundary_name':
                do_units = temp_gdf.iloc[-1]['LND_6_percentProtectedArea'] != 0
            most_recent.append(temp_gdf.copy())
    result = pd.concat([so_far_df] + most_recent, axis=0)
    result[['geo_id', 'geo_level', 'geo_name', 'geo_parent_name', 'LND_6_percentProtectedArea']].to_csv(OUTPUT_FILENAME)

In [ ]:
processedcities = pd.read_csv(OUTPUT_FILENAME)
processedcities

In [ ]:
# delete temporary files
for ext in ('shp', 'shx'):
    os.remove('{0}.{1}'.format(WDPA_TEMPFILENAME, ext))

# Merge with indicator table

In [ ]:
# read indicator table
cities_indicators = pd.read_csv(aws_s3_dir +'/'+ indicators_file_aws)
cities_indicators

In [ ]:
def merge_indicators(indicator_table, new_indicator_table, indicator_name):
    if indicator_name in indicator_table.columns:
        print("replace with new calculations")
        indicator_table.drop(indicator_name, inplace=True, axis=1)
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    else:
        print("add new indicators")
        cities_indicators_df = indicator_table.merge(new_indicator_table[["geo_id",indicator_name]], 
                                                     on='geo_id', 
                                                     how='left')
    return(cities_indicators_df)

In [ ]:
cities_indicators_merged = merge_indicators(indicator_table = cities_indicators,
                                            new_indicator_table = processedcities,
                                            indicator_name = 'LND_6_percentProtectedArea')

In [ ]:
cities_indicators_merged

## Upload in aws s3

In [ ]:
# upload to aws
key_data = indicators_file_aws
cities_indicators_merged.to_csv(
    f"s3://{bucket_name}/{key_data}",
    index=False,
    storage_options={
        "key": aws_key,
        "secret": aws_secret
    },
)

In [ ]:
# make it public
object_acl = s3.ObjectAcl(bucket_name,key_data)
response = object_acl.put(ACL='public-read')